In [1]:
filepath = '/Users/simonw/Dropbox/Backups/simonwillison-net-from-wayback-machine/wayback-machine-simonwillison-net-6-oct-2017.tar.gz'

In [2]:
import tarfile
tar = tarfile.open(filepath, 'r:gz')

In [3]:
paths = [info.name for info in tar if info.isreg()]

In [4]:
len(paths)

19146

In [5]:
paths[0]

'simonwillison.net/all.css'

In [6]:
import re
r = re.compile(r'^simonwillison\.net/(\d{4})/(\w+)/(\d+)/([^/]+)/index\.html')

In [9]:
item_paths = set('/' + p.replace('index.html', '').split('/', 1)[1] for p in paths if r.match(p))

In [11]:
len(item_paths), list(item_paths)[0]

(7358, '/2004/Jul/22/magical/')

In [12]:
MYPROJECT = '/Users/simonw/Dropbox/Development/simonwillisonblog/'
import os, sys
sys.path.insert(0, MYPROJECT)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings")
import django
django.setup()

In [13]:
from blog.models import Entry, Blogmark, Quotation

In [71]:
db_paths = set()
for klass in (Entry, Blogmark, Quotation):
    for obj in klass.objects.all():
        url = obj.get_absolute_url()
        db_paths.add(str(url))

In [72]:
missing = [p for p in item_paths if p not in db_paths and '#' not in p]

In [74]:
len(missing)

225

In [45]:
def extract_comments(soup):
    comments = []
    for div in soup.findAll('div', {'class': 'comment'}):
        id = div['id'].replace('c', '')
        footer = div.find('p', {'class': 'commentFooter'})
        if len(footer.findAll('a')) == 2:
            author = footer.find('a').text.strip()
            author_href = footer.find('a')['href']
            footer.find('a').extract()
        else:
            author = footer.text.rsplit('-', 2)[0].strip()
            author_href = None
        datestring = footer.text.rsplit('-', 2)[1].strip()
        footer.extract()
        comments.append({
            'id': id,
            'author': author,
            'author_href': author_href,
            'datetime': str(parser.parse(datestring)),
            'html': u''.join(map(unicode, div.contents)).strip(),
        })
    return comments


In [39]:
from dateutil import parser
from BeautifulSoup import BeautifulSoup as Soup
from HTMLParser import HTMLParser

In [46]:
def soup_to_item(soup):
    div = soup.find('div', {'class': 'entry entryPage'})
    footer = div.find('div', {'class': 'entryFooter'})
    datetime = str(parser.parse(footer.find('a').text + ' ' + footer.text.split('at ')[1]))
    footer.extract()
    d = {
        'datetime': datetime,
        'tags': [a.text for a in soup.findAll('a', {'rel': 'tag'})],
        'comments': extract_comments(soup),
    }
    # Is it a blogmark, an entry or a quotation?
    type = None
    if div.find('h2'):
        type = 'entry'
        d.update({
            'body': u''.join(map(unicode, div.contents)).strip(),
            'title': HTMLParser().unescape(div.find('h2').text)
        })
    elif div.find('blockquote'):
        type = 'quotation'
        d.update({
            'quotation': HTMLParser().unescape(div.find('blockquote').find('p').text),
            'source': HTMLParser().unescape(div.find('a').text),
            'source_url': div.find('a')['href'],
        })
    else:
        type = 'blogmark'
        a = div.find('a')
        link_url = a['href']
        link_title = HTMLParser().unescape(a.text)
        a.extract()
        via = div.find('a', title=True)
        via_url = None
        via_title = None
        if via:
            via_url = via['href']
            via_title = via['title']
            via.extract()
        commentary = HTMLParser().unescape(div.find('p').text or '')
        if commentary.startswith('() '):
            commentary = commentary[3:]
        if commentary.startswith('. '):
            commentary = commentary[2:]
        d.update({
            'link_url': link_url,
            'link_title': link_title,
            'via_url': via_url,
            'via_title': via_title,
            'commentary': commentary,
        })
    d['type'] = type
    return d


In [ ]:
items = []
i = 0
for path in missing:
    html = tar.extractfile('simonwillison.net%sindex.html' % path).read()
    soup = Soup(html)
    item = soup_to_item(soup)
    item['slug'] = [b for b in path.split('/') if b][-1]
    items.append(item)
    i += 1

In [55]:
len(items)

225

In [56]:
import json
open('/tmp/missing-content.json', 'w').write(json.dumps(items, indent=2))

In [57]:
!cat /tmp/missing-content.json | pbcopy

In [51]:
!subl /tmp/missing-content.json

In [75]:
len(items)

225

In [77]:
items[-1]

{'body': u'<h2>JSP bits and pieces</h2>\n<p>I\u2019ve been reading up on the Jakarta Struts <acronym title="Model View Controller">MVC</acronym> framework, courtesy of <a href="http://www.brunningonline.net/simon/blog/archives/000357.html" title="JSPs, the MVC model, and Struts">Simon Brunning</a>. <a href="http://www-106.ibm.com/developerworks/java/library/j-struts/index.html#h1">Struts, an open-source MVC implementation</a> is a great starting point, and the ONJava <a href="http://www.onjava.com/onjava/jsp_servlets/">JSP/JSTL series</a> (also found <a href="http://www.brunningonline.net/simon/blog/archives/000512.html#000512" title="O\'Reilly Struts article">via Simon</a>) were very informative as well.</p>',
 'comments': [],
 'datetime': '2002-11-01 14:56:00',
 'slug': 'jspbitsandpieces',
 'tags': [],
 'title': u'JSP bits and pieces',
 'type': 'entry'}